In [24]:
def loadlist(filename):
    with open(filename, 'r') as f:
        lines = []
        for line in f:
            lines.append(line[:-1])
    return(lines)

In [25]:
linky = loadlist("www.filmovyprehled.cz_all.txt")

In [31]:
linky[0:10]

['https://www.filmovyprehled.cz/en/person/86426/michal-vican',
 'https://www.filmovyprehled.cz/cs/person/84040/boris-krajny',
 'https://www.filmovyprehled.cz/cs/person/27613/jiri-hejl',
 'https://www.filmovyprehled.cz/cs/kalendar/detail/vsetinsky-filmovy-maraton',
 'https://www.filmovyprehled.cz/cs/film/197143/krehka-identita',
 'https://www.filmovyprehled.cz/cs/person/92640/mirage-production-service',
 'https://www.filmovyprehled.cz/en/person/5210/tamir-ment',
 'https://www.filmovyprehled.cz/en/person/24023/sophie-animation-co-ltd',
 'https://www.filmovyprehled.cz/cs/person/30140/alos-neuman',
 'https://www.filmovyprehled.cz/cs/person/76111/lukas-ujcik']

In [28]:
linky_filmy = [x for x in linky if "/cs/film/" in x]

In [30]:
linky_filmy[0:10]

['https://www.filmovyprehled.cz/cs/film/197143/krehka-identita',
 'https://www.filmovyprehled.cz/cs/film/402101/famu-v-kine-01',
 'https://www.filmovyprehled.cz/cs/film/102314/zimni-vila',
 'https://www.filmovyprehled.cz/cs/film/401931/barbora-polakova-krosna',
 'https://www.filmovyprehled.cz/cs/film/395473/jeji-pastorkyna',
 'https://www.filmovyprehled.cz/cs/film/397657/zacatek-dlouheho-podzimu',
 'https://www.filmovyprehled.cz/cs/film/397398/vrak',
 'https://www.filmovyprehled.cz/cs/film/395884/hvezda-z-posledni-stace',
 'https://www.filmovyprehled.cz/cs/film/397649/uzavreny-okruh',
 'https://www.filmovyprehled.cz/cs/film/396286/stenata']

## Začínáme scrapovat

In [32]:
import requests
from bs4 import BeautifulSoup

### Nejdřív zjistíme, co všechno budeme scrapovat

In [114]:
filmy_benchmark = ["kolja","pelisky","postriziny","lazarova","spalovac","dostavenicko-ve-mlynici"]
linky_benchmark = []
for l in linky_filmy:
    for f in filmy_benchmark:
        if f in l:
            linky_benchmark.append(l)
linky_benchmark

['https://www.filmovyprehled.cz/cs/film/396753/marketa-lazarova',
 'https://www.filmovyprehled.cz/cs/film/9013/kolja',
 'https://www.filmovyprehled.cz/cs/film/396780/spalovac-mrtvol',
 'https://www.filmovyprehled.cz/cs/film/397282/postriziny',
 'https://www.filmovyprehled.cz/cs/film/395109/dostavenicko-ve-mlynici',
 'https://www.filmovyprehled.cz/cs/film/37635/pelisky']

In [103]:
def jake_jsou_detaily(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    item_details = soup.find_all(class_='item-details')
    h3_texts = []
    for item_detail in item_details:
        h3_tags = item_detail.find_all('h3')
        for h3_tag in h3_tags:
            h3_texts.append(h3_tag.get_text())
    return(h3_texts)

In [117]:
detaily = []
for l in linky_benchmark:
    nove_detaily = jake_jsou_detaily(l)
    for n in nove_detaily:
        if n not in detaily:
            detaily.append(n)
detaily

['Země původu',
 'Copyright',
 'Rok výroby',
 'Premiéra',
 'Minutáž',
 'Režie',
 'Kategorie',
 'Žánr',
 'Typologie',
 'Originální název',
 'Český název',
 'Anglický název',
 'Souběžný název',
 'Souběžný anglický název']

## Scrapování filmů

In [118]:
def scrape_film(url):

    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    atributy = {}
    
    ## Detaily – země původu, copyright…
    
    details_div = soup.find('div', {'class': 'item-details'})
    details = []
    for div in details_div.find_all('div'):
        details.append(div.text)
    for x in detaily:
        for y in details:
            if x in y:
                y = y.replace(x,"")
                atributy[x] = y
                
    return(atributy)

In [121]:
scrape_film("https://www.filmovyprehled.cz/cs/film/396753/marketa-lazarova")

{'Země původu': 'Československo',
 'Copyright': '1967',
 'Rok výroby': '1965—1967',
 'Premiéra': '24. 11. 1967',
 'Minutáž': '160 min',
 'Režie': 'František Vláčil'}